# RAG (검색 증강 생성)

**RAG(Retrieval-Augmented Generation)**는 대규모 언어 모델(LLM)이 **외부의 지식(데이터)**을 참고하여 더 정확하고 풍부한 답변을 생성하도록 만드는 기술입니다.

쉽게 비유하자면, **"시험을 볼 때 교과서를 펼쳐 놓고 답을 찾는 오픈 북 테스트(Open-book Test)"**와 같습니다.

---

### **왜 RAG가 필요한가요?**
LLM(ChatGPT 등)은 다음과 같은 한계:
1.  **최신 정보 부재:** 학습 시점 이후의 정보는 알지 못합니다.
2.  **환각 현상 (Hallucination):** 모르는 내용도 사실인 것처럼 거짓말을 할 수 있습니다.
3.  **사내/비공개 데이터 접근 불가:** 기업 내부 문서나 개인적인 자료는 학습하지 않았습니다.

**👉 RAG는 외부 데이터베이스에서 "정답의 근거"를 찾아와서 LLM에게 제공함으로써 이 문제들을 해결합니다.**

---

### **RAG의 작동 원리 (3단계)**

1.  **검색 (Retrieval):** 사용자의 질문과 관련된 문서를 벡터 DB(Vector DB) 등에서 찾아옵니다.
2.  **증강 (Augmented):** 찾아온 정보를 프롬프트에 끼워 넣어(Context) 질문을 보강합니다.
3.  **생성 (Generation):** LLM은 보강된 정보를 바탕으로 신뢰할 수 있는 답변을 생성합니다.



In [1]:
!pip install langchain_openai langchain_core

  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.4 MB/s  0:00:00
Using cached jsonpointer-3.0.0-py2.py3-none-any.whl (7.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 51.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11/11 [langchain_openai][langchain_core]


In [2]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

LangChain 없이 직접 모델을 바꾸려면, 모델마다 API 사용 방식이 달라서 각각의 문서를 따로 찾아봐야 한다는 어려움이 있습니다.

예를 들어:

- Gemini API: https://ai.google.dev/gemini-api/docs?hl=ko
- OpenAI API: https://platform.openai.com/docs/guides/text”


In [3]:
import getpass
import os

# Prompt the user for the OpenAI API key securely
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

In [13]:
# 1. 모델 준비
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)            # temperature는 0으로 설정 높을 수록 답변 다양화(ex:0.2~0.5)

# 2. 우리 가게만의 지식 (Context) - 아직 DB 없이 변수로 직접 정의
context = """
[맛있는 김밥집 메뉴판]
1. 원조 김밥: 3,000원 - 40년 전통의 맛
2. 치즈 폭탄 김밥: 4,500원 - 모짜렐라 치즈가 듬뿍
3. 불닭 김밥: 5,000원 - 아주 매움, 맵찔이 금지
* 영업 시간: 오전 10시 ~ 오후 10시 (매주 월요일 휴무)
* 주문 마감시간 오후 8시
"""

In [14]:
# 3. 프롬프트 템플릿 만들기
# RAG의 핵심: {context} 자리에 검색된 문서를 넣는 것!
template = """
당신은 친절한 김밥집 점원입니다.
아래 [참고 정보]를 바탕으로 손님의 [질문]에 답변해주세요.
정보에 없는 내용은 "잘 모르겠습니다"라고 답하세요.

[참고 정보]
{context}

[질문]: {question}
"""

prompt = PromptTemplate.from_template(template)

In [15]:
# 4. LangChain의 파이프라인: LangChain Expression Language(LCEL)
# 프롬프트 -> LLM -> 문자열 변환
chain = prompt | llm | StrOutputParser()

In [16]:
# 5. 실행 및 결과 확인
question = "불닭 김밥 얼마인가요? 그리고 지금 저녁 9시인데 가도 되나요?"

response = chain.invoke({"context": context, "question": question})

print("=== AI 답변 ===")
print(response)

=== AI 답변 ===
불닭 김밥은 5,000원입니다. 하지만 지금 저녁 9시라서 주문 마감시간인 오후 8시가 지나버렸습니다. 그래서 지금은 주문하실 수 없습니다. 다음에 방문해 주세요!
